# Eight compared methods: IVW, dIVW, RAPS, Egger, MRMix, Weighted-median,  Weighted-mode, cML-MA. 

# Install packages

In [2]:
# TwosampleMR
# install.packages("remotes")
# remotes::install_github("MRCIEU/TwoSampleMR")

# dIVW
# install.packages("devtools")
# devtools::install_github("tye27/mr.divw")

# MRcML
# devtools::install_github("xue-hr/MRcML")

# MRMix
# devtools::install_github("gqi/MRMix")

# Run the eight compared MR methods

In [1]:
# 26 exposures
ts1 = c("AD", "ASD", "Daytime_Sleepiness", "Height_UKB",  "Intelligence", "RA",      
        "T2D", "Alcohol", "BMI", "Depression", "IBD", "MDD", "SCZ", "Angina", 
        "CAD", "HBP", "Income", "NEB", "Smoking", "Urate", "Anorexia", 
        "CD", "Height_GIANT", "Insomnia", "Neuroticism", "SWB")

# five negative control outcomes
ts2 = c("Hair_Light_Brown", "Hair_Dark_Brown",  "Hair_Black",    "Hair_Blonde",  "Tanning")

Threshold = 5e-08 # note that the default IV threshold for the eight methods is 5e-08


## IVW

In [2]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
    
    # reading in data
    cat("*********************************\n")
    cat("Pair: ", exposure,"~", outcome," ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
    
    # IVW
    res.IVW <- try(TwoSampleMR::mr_ivw_fe(dat$b.exp[indx], dat$b.out[indx], dat$se.exp[indx], dat$se.out[indx]))
    cat("IVW: beta.hat=", res.IVW$b, " se=", res.IVW$se, "pval=", res.IVW$pval, "\n")
    
    if(inherits(res.IVW,"try-error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "IVW", Threshold = Threshold,
                           nsnp = nsnp, beta =  res.IVW$b, se = res.IVW$se, pval = res.IVW$pval),
                "NC_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    
  }
}
print(proc.time()-start0)

*********************************
Pair:  AD ~ Hair_Light_Brown  IVW: beta.hat= 0.003039072  se= 0.01545384 pval= 0.8440977 
*********************************
Pair:  AD ~ Hair_Dark_Brown  IVW: beta.hat= -0.008688257  se= 0.01545384 pval= 0.5739749 
*********************************
Pair:  AD ~ Hair_Black  IVW: beta.hat= 0.002690524  se= 0.01545384 pval= 0.8617863 
*********************************
Pair:  AD ~ Hair_Blonde  IVW: beta.hat= 0.01861406  se= 0.01545384 pval= 0.2283986 
*********************************
Pair:  AD ~ Tanning  IVW: beta.hat= 0.009109343  se= 0.01560095 pval= 0.5592899 
*********************************
Pair:  ASD ~ Hair_Light_Brown  *********************************
Pair:  ASD ~ Hair_Dark_Brown  *********************************
Pair:  ASD ~ Hair_Black  *********************************
Pair:  ASD ~ Hair_Blonde  *********************************
Pair:  ASD ~ Tanning  *********************************
Pair:  Daytime_Sleepiness ~ Hair_Light_Brown  IVW: beta.hat= 0.

## dIVW

In [3]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
    
   # reading in data
    cat("*********************************\n")
    cat("Pair: ", exposure,"~", outcome," ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
    
    ## dIVW
    lambda.opt =  mr.divw::mr.eo(0, dat$b.exp[indx], dat$b.out[indx],
                                  dat$se.exp[indx], dat$se.out[indx],
                                  pval.selection=dat$pval.exp[indx])$lambda.opt
    res.dIVW = mr.divw::mr.divw(dat$b.exp[indx], dat$b.out[indx],
                                dat$se.exp[indx], dat$se.out[indx],
                                pval.selection=dat$pval.exp[indx], lambda=lambda.opt)
    cat("dIVW: beta.hat=", res.dIVW$beta.hat, " se=", res.dIVW$beta.se, "\n")
      
    
    if(inherits(res.dIVW,"try-error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "dIVW", Threshold = Threshold,
                           nsnp = nsnp, beta = res.dIVW$beta.hat, se = res.dIVW$beta.se, 
                           pval = pchisq(res.dIVW$beta.hat^2/res.dIVW$beta.se^2, 1, lower.tail = F)),
                "NC_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    
  }
}
print(proc.time()-start0)

*********************************
Pair:  AD ~ Hair_Light_Brown  dIVW: beta.hat= 0.00309123  se= 0.01571961 
*********************************
Pair:  AD ~ Hair_Dark_Brown  dIVW: beta.hat= -0.008837371  se= 0.01572348 
*********************************
Pair:  AD ~ Hair_Black  dIVW: beta.hat= 0.002736701  se= 0.01571949 
*********************************
Pair:  AD ~ Hair_Blonde  dIVW: beta.hat= 0.01893352  se= 0.01573932 
*********************************
Pair:  AD ~ Tanning  dIVW: beta.hat= 0.009265685  se= 0.01587351 
*********************************
Pair:  ASD ~ Hair_Light_Brown  *********************************
Pair:  ASD ~ Hair_Dark_Brown  *********************************
Pair:  ASD ~ Hair_Black  *********************************
Pair:  ASD ~ Hair_Blonde  *********************************
Pair:  ASD ~ Tanning  *********************************
Pair:  Daytime_Sleepiness ~ Hair_Light_Brown  dIVW: beta.hat= 0.04612366  se= 0.03131321 
*********************************
Pair:  Daytime_

## RAPS

In [4]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
    
   # reading in data
    cat("*********************************\n")
    cat("Pair: ", exposure,"~", outcome," ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
      
    ##  RAPS
    raps.df = data.frame(beta.exposure = dat$b.exp[indx], beta.outcome = dat$b.out[indx],
                         se.exposure = dat$se.exp[indx], se.outcome = dat$se.out[indx])
    res.raps <- try(suppressWarnings(mr.raps::mr.raps(raps.df, diagnostics=F)))
    cat("RAPS: beta.hat=", res.raps$beta.hat, " se=", res.raps$beta.se, "\n")
    
    if(inherits(res.raps,"try-error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "RAPS", Threshold = Threshold,
                           nsnp = nsnp, beta =  res.raps$beta.hat, se = res.raps$beta.se,
                           pval = pchisq(res.raps$beta.hat^2/res.raps$beta.se^2, 1, lower.tail = F)),
                "NC_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    

    
  }
}
print(proc.time()-start0)

*********************************
Pair:  AD ~ Hair_Light_Brown  RAPS: beta.hat= 0.008274147  se= 0.016086 
*********************************
Pair:  AD ~ Hair_Dark_Brown  RAPS: beta.hat= -0.01011918  se= 0.01712403 
*********************************
Pair:  AD ~ Hair_Black  RAPS: beta.hat= 0.004439251  se= 0.01606367 
*********************************
Pair:  AD ~ Hair_Blonde  RAPS: beta.hat= 0.01328635  se= 0.02369143 
*********************************
Pair:  AD ~ Tanning  RAPS: beta.hat= 0.0001497438  se= 0.02817929 
*********************************
Pair:  ASD ~ Hair_Light_Brown  *********************************
Pair:  ASD ~ Hair_Dark_Brown  *********************************
Pair:  ASD ~ Hair_Black  *********************************
Pair:  ASD ~ Hair_Blonde  *********************************
Pair:  ASD ~ Tanning  *********************************
Pair:  Daytime_Sleepiness ~ Hair_Light_Brown  RAPS: beta.hat= 0.0359696  se= 0.05061721 
*********************************
Pair:  Daytime_Sl

## Egger

In [5]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
    
    # reading in data
    cat("*********************************\n")
    cat("Pair: ", exposure,"~", outcome," ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
    
    ## Egger
    res.egger <- try(TwoSampleMR::mr_egger_regression(dat$b.exp[indx], dat$b.out[indx],
                                                      dat$se.exp[indx], dat$se.out[indx]))
    cat("Egger: beta.hat=", res.egger$b, " se=", res.egger$se, "pval=", res.egger$pval, "\n")
    
    
    if(inherits(res.egger,"error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "Egger", Threshold = Threshold,
                           nsnp = nsnp, beta =  res.egger$b, se = res.egger$se, pval = res.egger$pval),
                "NC_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    
  }
}
print(proc.time()-start0)

*********************************
Pair:  AD ~ Hair_Light_Brown  Egger: beta.hat= 0.06738147  se= 0.1335556 pval= 0.627496 
*********************************
Pair:  AD ~ Hair_Dark_Brown  Egger: beta.hat= -0.07196303  se= 0.1512316 pval= 0.6469067 
*********************************
Pair:  AD ~ Hair_Black  Egger: beta.hat= -0.02340052  se= 0.1274727 pval= 0.858916 
*********************************
Pair:  AD ~ Hair_Blonde  Egger: beta.hat= 0.05910276  se= 0.2032635 pval= 0.7786235 
*********************************
Pair:  AD ~ Tanning  Egger: beta.hat= -0.2167463  se= 0.2497758 pval= 0.410797 
*********************************
Pair:  ASD ~ Hair_Light_Brown  *********************************
Pair:  ASD ~ Hair_Dark_Brown  *********************************
Pair:  ASD ~ Hair_Black  *********************************
Pair:  ASD ~ Hair_Blonde  *********************************
Pair:  ASD ~ Tanning  *********************************
Pair:  Daytime_Sleepiness ~ Hair_Light_Brown  Egger: beta.hat= 0

## MRMix

In [6]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
    
    # reading in data
    cat("*********************************\n")
    cat("Pair: ", exposure,"~", outcome, " ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
    
    ## MRmix
    res.MRMix = try(MRMix::MRMix(dat[indx,]$b.exp, dat[indx,]$b.out, dat[indx,]$se.exp, dat[indx,]$se.out))
    cat("MRMix: beta.hat=", res.MRMix$theta, " se=", res.MRMix$SE_theta, "pval=", res.MRMix$pvalue_theta, "\n")
    
    if(inherits(res.MRMix,"try-error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method = "MRMix", Threshold = Threshold,
                           nsnp = nsnp, beta = res.MRMix$theta, se = res.MRMix$SE_theta, pval = res.MRMix$pvalue_theta),
                "NC_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    
  }
}
print(proc.time()-start0)

*********************************
Pair:  AD ~ Hair_Light_Brown  MRMix: beta.hat= 0  se= 0.03872263 pval= 1 
*********************************
Pair:  AD ~ Hair_Dark_Brown  MRMix: beta.hat= -0.02  se= 0.168699 pval= 0.9056285 
*********************************
Pair:  AD ~ Hair_Black  MRMix: beta.hat= 0.005  se= 0.0168703 pval= 0.7669407 
*********************************
Pair:  AD ~ Hair_Blonde  MRMix: beta.hat= -0.02  se= 0.02472732 pval= 0.4186175 
*********************************
Pair:  AD ~ Tanning  MRMix: beta.hat= -0.03  se= 0.02448928 pval= 0.2205654 
*********************************
Pair:  ASD ~ Hair_Light_Brown  *********************************
Pair:  ASD ~ Hair_Dark_Brown  *********************************
Pair:  ASD ~ Hair_Black  *********************************
Pair:  ASD ~ Hair_Blonde  *********************************
Pair:  ASD ~ Tanning  *********************************
Pair:  Daytime_Sleepiness ~ Hair_Light_Brown  MRMix: beta.hat= -0.03  se= 0.1053237 pval= 0.775769

## Weighted-median

In [8]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
    
    # reading in data
    cat("*********************************\n")
    cat("Pair: ", exposure,"~", outcome, " ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
    
    ## MRmix
    res.median <- try(TwoSampleMR::mr_weighted_median(dat$b.exp[indx], dat$b.out[indx], dat$se.exp[indx], dat$se.out[indx]))
    cat("Weighted-median: beta.hat=", res.median$b, " se=", res.median$se, "pval=", res.median$pval, "\n")
    
    if(inherits(res.median,"try-error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "Weighted-median", Threshold = Threshold,
                           nsnp = nsnp, beta = res.median$b, se = res.median$se, pval = res.median$pval),
                "NC_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    
  }
}
print(proc.time()-start0)

*********************************
Pair:  AD ~ Hair_Light_Brown  Weighted-median: beta.hat= 0.01786856  se= 0.02052257 pval= 0.3839298 
*********************************
Pair:  AD ~ Hair_Dark_Brown  Weighted-median: beta.hat= -0.01964828  se= 0.02101833 pval= 0.3498828 
*********************************
Pair:  AD ~ Hair_Black  Weighted-median: beta.hat= 0.02068743  se= 0.02010376 pval= 0.3034643 
*********************************
Pair:  AD ~ Hair_Blonde  Weighted-median: beta.hat= -0.005344507  se= 0.02268708 pval= 0.8137625 
*********************************
Pair:  AD ~ Tanning  Weighted-median: beta.hat= -0.009255771  se= 0.02312226 pval= 0.6889378 
*********************************
Pair:  ASD ~ Hair_Light_Brown  *********************************
Pair:  ASD ~ Hair_Dark_Brown  *********************************
Pair:  ASD ~ Hair_Black  *********************************
Pair:  ASD ~ Hair_Blonde  *********************************
Pair:  ASD ~ Tanning  *********************************
Pai

## Weighted-mode

In [9]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
    
    # reading in data
    cat("*********************************\n")
    cat("Pair: ", exposure,"~", outcome, " ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next

    res.mode <- try(TwoSampleMR::mr_weighted_mode(dat$b.exp[indx], dat$b.out[indx], dat$se.exp[indx], dat$se.out[indx]))
    cat("Weighted-mode: beta.hat=", res.mode$b, " se=", res.mode$se, "pval=", res.mode$pval, "\n")
    
    if(inherits(res.mode,"try-error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "Weighted-mode", Threshold = Threshold,
                           nsnp = nsnp, beta = res.mode$b, se = res.mode$se, pval = res.mode$pval),
                "NC_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 

    
  }
}
print(proc.time()-start0)


*********************************
Pair:  AD ~ Hair_Light_Brown  Weighted-mode: beta.hat= 0.02596796  se= 0.02688932 pval= 0.3594082 
*********************************
Pair:  AD ~ Hair_Dark_Brown  Weighted-mode: beta.hat= -0.02788219  se= 0.03238324 pval= 0.4115848 
*********************************
Pair:  AD ~ Hair_Black  Weighted-mode: beta.hat= 0.02121308  se= 0.03061583 pval= 0.5058695 
*********************************
Pair:  AD ~ Hair_Blonde  Weighted-mode: beta.hat= -0.0193032  se= 0.02903513 pval= 0.5228325 
*********************************
Pair:  AD ~ Tanning  Weighted-mode: beta.hat= -0.01548814  se= 0.03098215 pval= 0.6291353 
*********************************
Pair:  ASD ~ Hair_Light_Brown  *********************************
Pair:  ASD ~ Hair_Dark_Brown  *********************************
Pair:  ASD ~ Hair_Black  *********************************
Pair:  ASD ~ Hair_Blonde  *********************************
Pair:  ASD ~ Tanning  *********************************
Pair:  Daytime_S

## cML-MA

In [ ]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
    
   # reading in data
    cat("*********************************\n")
    cat("Pair: ", exposure,"~", outcome, " ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
    
    ## 3. cML-MA
    final_dat = dat[indx,]
    n= min(median(1/final_dat$se.exp^2), median(1/final_dat$se.out^2))
    cML_result = try(suppressWarnings(MRcML::mr_cML_DP(final_dat$b.exp,
                                                       final_dat$b.out,
                                                       final_dat$se.exp,
                                                       final_dat$se.out,
                                                       n = n,
                                                       random_start = 10,
                                                       random_start_pert = 10,
                                                       random_seed = 1,
                                                       num_pert = 200)))
    cat("cML-MA: beta.hat=", cML_result$MA_BIC_DP_theta, " se=", cML_result$MA_BIC_DP_se, "pval=", cML_result$MA_BIC_DP_p, "\n")
    
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "cML-MA", Threshold = Threshold,
                           nsnp = nsnp, beta =  cML_result$MA_BIC_DP_theta, se = cML_result$MA_BIC_DP_se, pval = cML_result$MA_BIC_DP_p),
                "NC_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    
    if(inherits(cML_result,"try-error")) next
    
  }
}
print(proc.time()-start0)